In [1]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
import os
from PIL import Image

# Define the dataset class
class CustomDataset(DatasetFolder):
    def __init__(self, root, transforms=None):
        super().__init__(root, loader=self.loader, extensions=('.jpg',), transform=transforms)

    def loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

# Define a function to parse annotation files
def parse_annotation(ann_path):
    with open(ann_path, 'r') as f:
        data = f.readline().strip().split()
        label = int(data[0])
        box = [float(x) for x in data[1:]]
    return label, box

# Define a function to collate data samples
def collate_fn(batch):
    return tuple(zip(*batch))

# Define transforms
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.ToTensor()
])

# Define the dataset paths
train_images_path = "./train/images"
train_ann_path = "./train/labels"
test_images_path = "./test/images"
test_ann_path = "./test/labels"

# Create datasets
train_dataset = CustomDataset(train_images_path, transforms=transforms)
test_dataset = CustomDataset(test_images_path, transforms=transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Define the Faster R-CNN model
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
model = FasterRCNN(backbone, num_classes=2, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)

# Define device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

FileNotFoundError: [Errno 2] No such file or directory: './train/images'

In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    model.train()
    header = 'Epoch: [{}]'.format(epoch)
    for i, (images, targets) in enumerate(data_loader):
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % print_freq == 0:
            print('{}\tBatch: [{}]\tLoss: {:.4f}'.format(header, i, losses.item()))

def evaluate(model, data_loader, device):
    model.eval()
    header = 'Test:'
    with torch.no_grad():
        for i, (images, targets) in enumerate(data_loader):
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(images)
            # You can perform evaluation metrics calculation here
            # For example, calculate mAP, precision, recall, etc.

def main():
    num_epochs = 10
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, train_loader, device, epoch)
        evaluate(model, test_loader, device)

if __name__ == "__main__":
    main()
